# Low-Level Design (LLD) of a Parking Lot System

Designing a Parking Lot System involves creating a structure that allows users to park vehicles, keep track of available spaces, handle parking fees, and manage parking for multiple vehicles efficiently.

### 1. Requirements

1. **Parking a Vehicle**:
    - The system should allow a vehicle to be parked in an available parking spot.
    - Vehicle types: Cars, Motorcycles, and potentially larger vehicles like Buses.

2. **Exiting the Parking Lot**:
    - A vehicle should be able to exit, and the parking spot should be freed up.

3. **Parking Fees**:
   - Calculate parking fees based on the time the vehicle was parked.
   - Different rates should be applied for different vehicle types.

4. **Parking Lot Management**:
   - Track available and occupied spots.
   - Allow users to query available parking spots for different vehicle types.
   - Handle overflows when the parking lot is full.

5. **Multiple Vehicle Types**:
   - The system should handle cars, motorcycles, and possibly other vehicle types (like buses, electric vehicles)

6. **Parking Lot with Multiple Levels**:
   - The parking lot may have multiple levels with different types of parking spaces

---

### 2. Constraints

1. The parking lot may have a limited number of spots for each type of vehicle.
2. The system should operate under the assumption that vehicles are identified uniquely.
3. A parking ticket is required to calculate fees and allow vehicle exit.

---

### 3. Identify Entities

1. **Vehicle**:
    - Represents a vehicle that can be parked. Each vehicle will have an ID, type (car, motorcycle, etc.), and entry/exit time

2. **ParkingSpot**:
    - Represents a parking spot. It should include information about the parking spot's type (compact, standard, etc.), whether it's available, and which vehicle is currently occupying it

3. **ParkingLot**:
   - Represents the entire parking lot with multiple levels and parking spots. It is responsible for managing the parking process (assigning and freeing up spots)

4. **ParkingTicket**:
   - Represents a parking ticket issued when a vehicle enters and exits. It holds details such as entry time, exit time, parking fee, and the vehicle associated with it

5. **ParkingFeeCalculator**:
   - Responsible for calculating parking fees based on the time spent and the type of vehicle.

### 4. Class Design

#### 4.1. Vehicle Classes

In [5]:
from abc import ABC
from enum import Enum


class VehicleType(Enum):
    CAR = 1
    MOTORCYCLE = 2
    TRUCK = 3
    BUS = 4
    ELECTRIC = 5  # Special category for EVs


class Vehicle(ABC):
    def __init__(self, vehicle_id: str, vehicle_type: VehicleType):
        self._vehicle_id = vehicle_id
        self._vehicle_type = vehicle_type

    @property
    def vehicle_id(self) -> str:
        return self._vehicle_id

    @property
    def vehicle_type(self) -> VehicleType:
        return self._vehicle_type


class Car(Vehicle):
    def __init__(self, vehicle_id: str):
        super().__init__(vehicle_id, VehicleType.CAR)


class Motorcycle(Vehicle):
    def __init__(self, vehicle_id: str):
        super().__init__(vehicle_id, VehicleType.MOTORCYCLE)


class Truck(Vehicle):
    def __init__(self, vehicle_id: str):
        super().__init__(vehicle_id, VehicleType.TRUCK)


class Bus(Vehicle):
    def __init__(self, vehicle_id: str):
        super().__init__(vehicle_id, VehicleType.BUS)


class ElectricVehicle(Vehicle):
    def __init__(self, vehicle_id: str):
        super().__init__(vehicle_id, VehicleType.ELECTRIC)


#### 4.2. ParkingSpot Class

In [7]:
from enum import Enum


class ParkingSpotType(Enum):
    COMPACT = "Compact"
    LARGE = "Large"
    MOTORCYCLE = "Motorcycle"
    ELECTRIC = "Electric"


class ParkingSpot:

    def __init__(self, spot_id: int, spot_type: ParkingSpotType):
        self._spot_id = spot_id
        self._spot_type = spot_type 
        self._is_occupied = False
        self._vehicle = None

    @property
    def spot_id(self) -> int:
        return self._spot_id

    @property
    def spot_type(self) -> ParkingSpotType:
        return self._spot_type

    @property
    def is_occupied(self) -> bool:
        return self._is_occupied

    @property
    def vehicle(self) -> Vehicle:
        return self._vehicle

    def assign_vehicle(self, vehicle: Vehicle) -> None:
        if self._is_occupied:
            raise SpotAlreadyOccupiedException(self._spot_id)
        self._is_occupied = True
        self._vehicle = vehicle

    def free_up(self) -> None:
        if not self._is_occupied:
            raise SpotNotOccupiedException(self._spot_id)
        self._is_occupied = False
        self._vehicle = None


#### 4.3. ParkingFloor Class

In [9]:
class ParkingFloor:
    
    def __init__(self, floor_id: int, max_spots: dict):
        self._floor_id = floor_id
        self._spots = {}
        self._max_spots = max_spots
    
    @property
    def floor_id(self) -> int:
        return self._floor_id

    def add_spot(self, spot: ParkingSpot) -> None:
        if spot.spot_type not in self._max_spots:
            raise SpotTypeNotFoundException(spot.spot_type)

        if spot.spot_type not in self._spots:
            self._spots[spot.spot_type] = []

        if len(self._spots[spot.spot_type]) >= self._max_spots[spot.spot_type]:
            raise ParkingFloorFullException(spot.spot_type)
        
        self._spots[spot.spot_type].append(spot)

    def get_available_spot(self, spot_type: ParkingSpotType) -> ParkingSpot:
        spot_list = self._spots.get(spot_type, [])
        for spot in spot_list:
            if not spot.is_occupied:
                return spot
        return None

    def release_spot(self, spot_id: int) -> None:
        for spot_list in self._spots.values():
            for spot in spot_list:
                if spot.spot_id == spot_id:
                    spot.free_up()
                    return
        raise SpotNotFoundException(f"Spot ID {spot_id} not found.")


#### 4.4. ParkingTicket Class

In [11]:
from datetime import datetime


class ParkingTicket:

    def __init__(self, ticket_id: str, vehicle: Vehicle, spot: ParkingSpot, floor_id: str):
        self._ticket_id = ticket_id
        self._vehicle = vehicle
        self._floor_id = floor_id
        self._spot = spot
        self._entry_time = datetime.now()
        self._exit_time = None
        
    @property
    def ticket_id(self) -> str:
        return self._ticket_id
    
    @property
    def floor_id(self) -> int:
        return self._floor_id
    
    @property
    def vehicle(self) -> Vehicle:
        return self._vehicle
    
    @property
    def spot(self) -> Spot:
        return self._spot

    def set_exit_time(self) -> None:
        self._exit_time = datetime.now()

    def calculate_fee(self, fee_calculator) -> float:
        if not self._exit_time:
            raise ParkingLotException("Exit time is not set.")
        parking_fee = fee_calculator.calculate_fee(self._entry_time, self._exit_time, self._vehicle.vehicle_type)
        return parking_fee


#### 4.5. ParkingFeeCalculator Class

In [13]:
from datetime import datetime

class ParkingFeeCalculator:
    
    def __init__(self):
        self.rates = {
            VehicleType.MOTORCYCLE.value: 5,
            VehicleType.CAR.value: 10,
            VehicleType.TRUCK.value: 15,
            VehicleType.BUS.value: 20,
            VehicleType.ELECTRIC.value: 8
        }

    def calculate_fee(self, entry_time: datetime, exit_time: datetime, vehicle: Vehicle) -> float:
        if entry_time >= exit_time:
            raise InvalidTimeRangeException(entry_time, exit_time)

        if vehicle.vehicle_type not in self.rates:
            raise InvalidVehicleTypeException(vehicle.vehicle_type)
        
        parking_duration = (exit_time - entry_time).total_seconds() / 3600
        rate = self.rates[vehicle.vehicle_type]
        return parking_duration * rate


#### 4.6. ParkingLot Class

In [15]:
class ParkingLot:
    
    def __init__(self, name: str, max_floor: int):
        self._name = name
        self._floors = []
        self._max_floor = max_floor
        self._vehicle_to_spot_map = {
            VehicleType.CAR: ParkingSpotType.COMPACT,
            VehicleType.MOTORCYCLE: ParkingSpotType.MOTORCYCLE,
            VehicleType.TRUCK: ParkingSpotType.LARGE,
            VehicleType.ELECTRIC: ParkingSpotType.ELECTRIC
        }

    def add_floor(self, floor: ParkingFloor) -> None:
        if len(self._floors) >= self._max_floor:
            raise ParkingLotFullException(self._max_floor)

        self._floors.append(floor)
        print(f"Floor {floor.floor_id} added to Parking Lot.")

    def park_vehicle(self, vehicle: Vehicle) -> None:
        for floor in self._floors:
            spot = floor.get_available_spot(self._vehicle_to_spot_map[vehicle.vehicle_type])  # Use getter method for type
            if spot:
                spot.assign_vehicle(vehicle)
                ticket = ParkingTicket(f"{vehicle.vehicle_id}_{spot.spot_id}", vehicle, spot, floor.floor_id)
                print(f"Vehicle {vehicle.vehicle_id} parked at Floor {floor.floor_id}, Spot {spot.spot_id}.")
                return ticket
    
        raise ParkingLotFullException(vehicle.vehicle_type)

    def release_spot(self, ticket: ParkingTicket) -> None:
        floor_id = ticket.floor_id
        spot_id = ticket.spot.spot_id
        for floor in self._floors:  # Corrected the reference to self._floors
            if floor.floor_id == floor_id:
                floor.release_spot(spot_id)
                ticket.set_exit_time()
                print(f"Vehicle {ticket.vehicle.vehicle_id} exited from Floor {floor_id}, Spot {spot_id}.")
                return

        raise ParkingFloorNotFoundException(floor_id)


### 5. Exception Handling

In [17]:
class ParkingLotException(Exception):
    pass


class SpotAlreadyOccupiedException(ParkingLotException):
    def __init__(self, spot_id):
        super().__init__(f"Parking spot {spot_id} is already occupied.")


class SpotNotOccupiedException(Exception):
    def __init__(self, spot_id: int):
        super().__init__(f"Parking spot {spot_id} is not occupied.")


class SpotNotFoundException(ParkingLotException):
    def __init__(self, spot_id):
        super().__init__(f"Parking spot {spot_id} does not exist.")


class SpotTypeNotFoundException(ParkingLotException):
    def __init__(self, spot_type):
        super().__init__(f"Spot type {spot_type} not found.")


class ParkingFloorNotFoundException(ParkingLotException):
    def __init__(self, floor_id):
        super().__init__(f"Parking floor {floor_id} does not exist.")
        
class ParkingFloorFullException(ParkingLotException):
    def __init__(self, spot_type):
        super().__init__(f"Parking floor is full for {spot_type} spots.")


class VehicleNotFoundException(ParkingLotException):
    def __init__(self, vehicle_id):
        super().__init__(f"Vehicle with ID {vehicle_id} is not found.")

        
class ParkingLotFullException(ParkingLotException):
    def __init__(self, vehicle_type):
        super().__init__(f"No available spots for vehicle type {vehicle_type}. The parking lot is full!")


class InvalidVehicleTypeException(ParkingLotException):
    def __init__(self, vehicle_type):
        super().__init__(f"Invalid vehicle type: {vehicle_type}. Please provide a valid type.")

        
class InvalidTimeRangeException(ParkingLotException):
    def __init__(self, entry_time, exit_time):
        super().__init__(f"Entry time ({entry_time}) must be before exit time ({exit_time}).")

### 6. Implementation

In [19]:
def main():
    parking_lot = ParkingLot(name="City Center Parking", max_floor=2)

    # Create and add floors
    floor1 = ParkingFloor(floor_id=1,  max_spots={ParkingSpotType.COMPACT: 1, ParkingSpotType.MOTORCYCLE: 1, ParkingSpotType.LARGE: 1})
    floor2 = ParkingFloor(floor_id=2, max_spots={ParkingSpotType.COMPACT: 1, ParkingSpotType.MOTORCYCLE: 1, ParkingSpotType.LARGE: 1})

    # Add spots to floors
    floor1.add_spot(ParkingSpot(spot_id=1, spot_type=ParkingSpotType.COMPACT))
    floor1.add_spot(ParkingSpot(spot_id=2, spot_type=ParkingSpotType.MOTORCYCLE))
    floor2.add_spot(ParkingSpot(spot_id=3, spot_type=ParkingSpotType.COMPACT))
    floor2.add_spot(ParkingSpot(spot_id=4, spot_type=ParkingSpotType.LARGE))

    # Add floors to parking lot
    parking_lot.add_floor(floor1)
    parking_lot.add_floor(floor2)

    try:
        # Park vehicles
        vehicle1 = Vehicle(vehicle_id="V001", vehicle_type=VehicleType.CAR)
        ticket1 = parking_lot.park_vehicle(vehicle1)

        vehicle2 = Vehicle(vehicle_id="V002", vehicle_type=VehicleType.MOTORCYCLE)
        ticket2 = parking_lot.park_vehicle(vehicle2)

        # Try to park when no spots are available
        vehicle3 = Vehicle(vehicle_id="V003", vehicle_type=VehicleType.TRUCK)
        parking_lot.park_vehicle(vehicle3)  # Raises exception if no spot is available
        
        try:
            vehicle4 = Vehicle(vehicle_id="V004", vehicle_type=VehicleType.ELECTRIC)
            ticket4 = parking_lot.park_vehicle(vehicle4)
        except ParkingLotFullException as err:
            print(err)

        try:
            vehicle5 = Vehicle(vehicle_id="V005", vehicle_type=VehicleType.TRUCK)
            ticket5 = parking_lot.park_vehicle(vehicle5)
        except ParkingLotFullException as err:
            print(err)

        # Release vehicles
        parking_lot.release_spot(ticket1)
        parking_lot.release_spot(ticket2)

    except ParkingLotException as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    main()


Floor 1 added to Parking Lot.
Floor 2 added to Parking Lot.
Vehicle V001 parked at Floor 1, Spot 1.
Vehicle V002 parked at Floor 1, Spot 2.
Vehicle V003 parked at Floor 2, Spot 4.
No available spots for vehicle type VehicleType.ELECTRIC. The parking lot is full!
No available spots for vehicle type VehicleType.TRUCK. The parking lot is full!
Vehicle V001 exited from Floor 1, Spot 1.
Vehicle V002 exited from Floor 1, Spot 2.
